## Imports

In [24]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten

In [25]:
df = pd.read_csv('map_template_popup_pin_kate_map - palm_beach.csv')

In [26]:
df.columns

Index(['full_address', 'developers', 'price', 'description', 'story_link'], dtype='object')

## Image URL collector

In [27]:
# Drop last n rows of a df. In case there are sums that need clipped
# df.drop(df.tail(n).index, inplace=True)


In [28]:
df['image_links'] = ''
links = []
for url in df['story_link']:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        images = soup.find_all('img')
        counter = 0
        for image in images:
            counter += 1
            if 'jpg' in image['src']:
                links.append(image['src'])
                if counter == len(images):
                    links.append('no image found')
                break
    except:
        links.append('no_image')

df['image_links'] = links

## Article link formatter

In [29]:
links

['https://s13.therealdeal.com/trd/m/up/2022/06/main-Manalapan-compound-poised-to-sell-for-Florida-record-175M-705x440.jpg',
 'https://s12.therealdeal.com/trd/m/up/2022/06/Oceanfront-Palm-Beach-estate-flips-for-86M-in-nine-months-705x466.jpg',
 'https://s13.therealdeal.com/trd/m/up/2022/05/Main_ivana-mansion-705x439.jpg',
 'https://s14.therealdeal.com/trd/m/up/2022/02/main-Todd-Glaser-partners-buy-waterfront-Palm-Beach-mansion-with-plans-to-re-list-it-for-115M-705x439.jpg',
 'https://s12.therealdeal.com/trd/m/up/2022/04/main-Non-waterfront-Palm-Beach-spec-home-on-Billionaires-Row-trades-for-48.5M-705x439.jpg',
 'https://s14.therealdeal.com/trd/m/up/2022/06/main_MIA_Walker-Palm-Beach-705x439.jpg',
 'https://s11.therealdeal.com/trd/m/up/2022/03/Royal-Real-Estate-Jordanian-Princess-sells-waterfront-Palm-Beach-mansion-705x431.jpg',
 'https://s13.therealdeal.com/trd/m/up/2022/03/main-MIA-Fiji-water-palm-beach-705x439.jpg',
 'https://s13.therealdeal.com/trd/m/up/2022/05/MIA-Todd-Glaser-flips-

In [30]:
df.description

0    The sale of this 16-acre estate sets a new rec...
1    This 18,000-square-foot oceanfront mansion sol...
2    This 12,352-square-foot waterfront mansion fet...
3    Spec home developer Todd Michael Glaser and hi...
4    This eight-bedroom, ten-bathroom spec mansion ...
5    This waterfront estate traded hands off-market...
6    Jordanian princess Alia Bing Hussein sold her ...
7    Fiji Water founder David Gilmour and his wife ...
8    Spec home developer Todd Michael Glaser's seco...
9    This waterfront Palm Beach estate flipped for ...
Name: description, dtype: object

In [31]:
df["short_description"] = df["description"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['description'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story_link"] + '">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0    <a href="https://therealdeal.com/miami/2022/06...
1    <a href="https://therealdeal.com/miami/2022/06...
2    <a href="https://therealdeal.com/miami/2022/05...
3    <a href="https://therealdeal.com/miami/2022/02...
4    <a href="https://therealdeal.com/miami/2022/04...
5    <a href="https://therealdeal.com/miami/2022/06...
6    <a href="https://therealdeal.com/miami/2022/03...
7    <a href="https://therealdeal.com/miami/2022/03...
8    <a href="https://therealdeal.com/miami/2022/05...
9    <a href="https://therealdeal.com/miami/2022/06...
Name: description_link, dtype: object

## Google Maps API Geolocater Setup

In [32]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [33]:
df['geo_address'] = df['full_address'] + ', Palm Beach, FL'
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## HTML popup formatter

In [40]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
#     developer = df['developers'].iloc[i]
    description = df['description_link'].iloc[i]
    image = df['image_links'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <img src={} width="256" height="156">'''.format(image) + '''<br>______________________________________<br>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    <strong>{}'''.format(description) + '''</strong></html>
    '''
    return html


### HTML reservoir

In [ ]:
#

## Map Maker

In [38]:
df.description_link

0    <a href="https://therealdeal.com/miami/2022/06...
1    <a href="https://therealdeal.com/miami/2022/06...
2    <a href="https://therealdeal.com/miami/2022/05...
3    <a href="https://therealdeal.com/miami/2022/02...
4    <a href="https://therealdeal.com/miami/2022/04...
5    <a href="https://therealdeal.com/miami/2022/06...
6    <a href="https://therealdeal.com/miami/2022/03...
7    <a href="https://therealdeal.com/miami/2022/03...
8    <a href="https://therealdeal.com/miami/2022/05...
9    <a href="https://therealdeal.com/miami/2022/06...
Name: description_link, dtype: object

In [45]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=11)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'New Development in Downtown Fort Lauderdale')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [ ]:
m.save('index.html')

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [ ]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [ ]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name  + cwd[-1]
print(final_name)